# Bagging 

## 📖 What Is Bagging?

Bagging—short for **bootstrap aggregating**—is an **ensemble** method that trains many independent copies of the same base learner on different bootstrap samples (random samples *with* replacement) and then averages their predictions.

The idea: **reduce variance** without increasing bias by “voting” across diverse models that each see slightly different data.

---

### How It Works

Conceptual algorithm: 
1. **Bootstrap** the training set $B$ times → $B$ equally-sized samples **with replacement**.
2. Train an **independent** (usually deep) tree on each sample.
3. **Aggregate** using majority vote for classification and mean/median for regression.
4. **Model selection** (or parameter tuning) using **out-of-bag** (OOB) errors.

---

### Why It Works


* Each tree “sees” a *different* slice of data → their errors are less correlated.
* Averaging cancels out noise ⇒ **lower variance** and better generalization.


---

### ⚙️ Popular Flavors

| Ensemble                                     | Extra Twist                                                                                 |
| -------------------------------------------- | ------------------------------------------------------------------------------------------- |
| **Plain BaggingClassifier / Regressor**      | Same features for every tree; randomness only from bootstrapped rows.                       |
| **Random Forest**                            | Adds column-sampling at every *split* to further de-correlate trees.                        |
| **Extra Trees (Extremely Randomized Trees)** | Uses the whole dataset but selects split thresholds at random—high bias, very low variance. |

---

### ⚠️ Pros and Cons

✅ **Parallel-friendly**: each model trains independently—easy to distribute.  
✅ Dramatically **reduces variance** of high-variance learners (e.g.\ full-depth trees).  
✅ Mitigates overfitting without heavy hyper-parameter tuning.  
✅ Naturally provides **out-of-bag** error estimates—no extra validation set needed.  

❌ Yields **large, memory-hungry** ensembles (hundreds of trees).  
❌ Adds little benefit if the base learner is already low-variance (e.g.\ regularized linear models).  
❌ Final model loses interpretability of a single tree; feature importance is more diffuse.  

---

Key Hyper-parametersa

| Symbol in `sklearn` | Meaning                          | Typical starter value |
| ------------------- | -------------------------------- | --------------------- |
| `n_estimators`      | number of bootstrapped trees $B$ | 100                   |
| `max_samples`       | rows per bootstrap sample        | 0.6 – 1.0             |
| `max_features`      | columns sent to each tree        | 1.0 (all)             |

---

> *Think of Bagging as crowd-sourcing your model: each member is noisy, but their average is calm and reliable.* --- Words of wisdom by ChatGPT 3o (again)



#### 3.3  Bagging (Bootstrap Aggregation)


Why it works: 


In [2]:
import numpy as np

rng = np.random.default_rng(0)   
# Examples with more data points:
sample_size = 200
X = rng.uniform(0.1, 0.9, size=(sample_size, 2))
y = np.zeros(sample_size, dtype=int)
mask1 = X[:, 0] + X[:, 1] > 1.1
mask2 = (~mask1) & (X[:, 0] - X[:, 1] > 0.3)
y[mask1] = 1
y[mask2] = 0
y[~(mask1 | mask2)] = 2


In [4]:
# ----- imports -------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import sklearn                                   # version check
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

import ipywidgets as widgets
from ipywidgets import interact


label_cmap = ListedColormap(["#1f77b4", "#ff7f0e", "#2ca02c"])

# ----- helper: fit & plot a bagging ensemble --------------------------------
def plot_bagging(max_features=1.0, n_estimators=100, max_depth=3):
    depth = None if max_depth == 0 else int(max_depth)

    # build BaggingClassifier with proper keyword for your scikit-learn version
    bag_kwargs = dict(
        n_estimators=int(n_estimators),
        max_features=max_features,
        bootstrap=True,
        oob_score=True,
        random_state=42,
    )
    tree = DecisionTreeClassifier(max_depth=depth, random_state=42)
    if sklearn.__version__ >= "1.4":
        bag_kwargs["estimator"] = tree
    else:                                      # pre-1.4 API
        bag_kwargs["base_estimator"] = tree

    bag = BaggingClassifier(**bag_kwargs).fit(X, y)

    # mesh grid over feature space
    x_min, x_max = X[:, 0].min() - 0.05, X[:, 0].max() + 0.05
    y_min, y_max = X[:, 1].min() - 0.05, X[:, 1].max() + 0.05
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 400),
        np.linspace(y_min, y_max, 400),
    )
    Z = bag.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # plot decision regions + data
    plt.figure(figsize=(6, 5))
    plt.contourf(xx, yy, Z, alpha=0.25, cmap=label_cmap)
    plt.scatter(X[:, 0], X[:, 1],
                c=y, cmap=label_cmap,
                edgecolors="k", s=80)

    depth_label = "None" if depth is None else depth
    plt.title(f"Bagging (Decision Trees): max_features={max_features:.2f}, "
              f"n_estimators={n_estimators}, max_depth={depth_label}\n"
              f"OOB accuracy ≈ {bag.oob_score_:.3f}")
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.tight_layout()
    plt.show()

# ----- interactive widgets --------------------------------------------------
interact(
    plot_bagging,
    max_features=widgets.FloatSlider(
        value=1.0, min=0.2, max=1.0, step=0.1,
        description="max_features (≤1.0)"
    ),
    n_estimators=widgets.IntSlider(
        value=100, min=10, max=300, step=10,
        description="n_estimators"
    ),
    max_depth=widgets.IntSlider(
        value=3, min=0, max=6, step=1,
        description="max_depth (0=unlimited)"
    ),
);


interactive(children=(FloatSlider(value=1.0, description='max_features (≤1.0)', max=1.0, min=0.2), IntSlider(v…